In [1]:
#Importing essential libraries
from sklearn.datasets import fetch_openml

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os 
# import cv2
import random 
import math
# import imblearn

from datetime import datetime

from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import pickle
# import tensorflow as tf
# from tensorflow import keras

In [2]:
pickle_in = open("X_train_rusb.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("y_train_rusb.pickle","rb")
y_train = pickle.load(pickle_in)

pickle_in = open("X_test_rusb.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("y_test_rusb.pickle","rb")
y_test = pickle.load(pickle_in)


In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:
scaler = StandardScaler()

In [5]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [6]:

from sklearn import preprocessing
from scipy.io.arff import loadarff
from sklearn.ensemble import RandomForestClassifier
import numpy as np

## Random Forest

In [7]:
# Fit to model and predict
rfc = RandomForestClassifier()
y_pred = rfc.fit(X_train, y_train).predict(X_test)

In [8]:
total_datapoints = X_test.shape[0]
mislabeled_datapoints = (y_test != y_pred).sum()
correct_datapoints = total_datapoints-mislabeled_datapoints
percent_correct = (correct_datapoints / total_datapoints) * 100

print("RandomForestClassifier results for CIC IDS 2017 n")
print("Total datapoints: %d\nCorrect datapoints: %d\nMislabeled datapoints: %d\nPercent correct: %.2f%%"
      % (total_datapoints, correct_datapoints, mislabeled_datapoints, percent_correct))

RandomForestClassifier results for CIC IDS 2017 n
Total datapoints: 255445
Correct datapoints: 206928
Mislabeled datapoints: 48517
Percent correct: 81.01%


In [9]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))


[[127782      0]
 [ 48517  79146]]
              precision    recall  f1-score   support

           0       0.72      1.00      0.84    127782
           1       1.00      0.62      0.77    127663

    accuracy                           0.81    255445
   macro avg       0.86      0.81      0.80    255445
weighted avg       0.86      0.81      0.80    255445

0.8100687036348333


## XGBoost

In [10]:
from xgboost import XGBClassifier
model = XGBClassifier(silent=0,
                     scale_pos_weight=1,
                     learning_rate=0.01,
                     colsample_bytree=0.6,
                     subsample=0.8,
                     objective='binary:logistic',
                     n_estimators=100,
                     reg_alpha=0.3,
                     max_depth=3,
                     gamma=1)

model.fit(X_train,y_train,verbose=True)

[09:11:52] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0.3, reg_lambda=1, scale_pos_weight=1, silent=0,
              subsample=0.8, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [11]:
preds = model.predict(X_test)

print(accuracy_score(y_test,preds))

0.9366478106833174


In [12]:
import xgboost as xgb
boost = xgb.XGBClassifier(max_depth=9,
                          subsample=0.9,
                          objective='multi:softmax',
                          num_class = 3,
                          min_child_weight=2,
                          colsample_bytree=0.7,
                          n_estimators=100,
                          learning_rate=0.08,
                          n_jobs = -1)
boost.fit(X_train,y_train)
boost_pred = boost.predict(X_test)


In [13]:
from sklearn.metrics import accuracy_score 
accuracy_score(y_test,boost_pred)

0.9130027990369747

In [14]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,boost_pred))  
print(classification_report(y_test,boost_pred))  
print(accuracy_score(y_test, boost_pred))


[[127768     14]
 [ 22209 105454]]
              precision    recall  f1-score   support

           0       0.85      1.00      0.92    127782
           1       1.00      0.83      0.90    127663

    accuracy                           0.91    255445
   macro avg       0.93      0.91      0.91    255445
weighted avg       0.93      0.91      0.91    255445

0.9130027990369747


## Bagging Classifier

In [15]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(n_estimators=5, warm_start=True,random_state=3141)

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [16]:
print("Accuracy score for Bagging classifier is:",accuracy_score(y_test,y_pred))

Accuracy score for Bagging classifier is: 0.7892148994891268


In [17]:
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))

[[125075   2707]
 [ 51137  76526]]
              precision    recall  f1-score   support

           0       0.71      0.98      0.82    127782
           1       0.97      0.60      0.74    127663

    accuracy                           0.79    255445
   macro avg       0.84      0.79      0.78    255445
weighted avg       0.84      0.79      0.78    255445

0.7892148994891268


## KNN

In [18]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier()
knn_model.fit(X_train,y_train.values.ravel())
knn_prediction = knn_model.predict(X_test)

In [19]:
print("Accuracy score for KNN:",accuracy_score(y_test,knn_prediction))

Accuracy score for KNN: 0.9847246961185382


In [20]:
print(confusion_matrix(y_test,knn_prediction))  
print(classification_report(y_test,knn_prediction))  
print(accuracy_score(y_test, knn_prediction))

[[126170   1612]
 [  2290 125373]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.98    127782
           1       0.99      0.98      0.98    127663

    accuracy                           0.98    255445
   macro avg       0.98      0.98      0.98    255445
weighted avg       0.98      0.98      0.98    255445

0.9847246961185382
